## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 9 enero

In [2]:
pd.__version__

'1.2.0'

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * estado_vacunacion_.csv , última versión del mismo con el acumulado de todos los informes publicados
  * estado_vacunacion_year-month-day.csv, con la versión del día en cuestión, solo con la información de ese día

In [3]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha el informe para el nombre del fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #y para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en formato ods

In [4]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [5]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210108.ods


In [6]:
datos_row

,Unnamed: 0,Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,140295,69445,0.494993,2021-01-07
1,Aragón,23715,9944,0.419313,2021-01-08
2,Asturias,23720,17843,0.752234,2021-01-08
3,Baleares,12840,4257,0.331542,2021-12-07
4,Canarias,27660,9572,0.346059,2021-01-08
5,Cantabria,11850,2334,0.196962,2021-01-08
6,Castilla y Leon,51390,28325,0.551177,2021-01-07
7,Castilla La Mancha,35550,12380,0.348242,2021-01-07
8,Cataluña,120545,38826,0.322087,2021-01-07
9,C. Valenciana,61225,15965,0.260759,2021-01-08


..copia para tratarlo

In [7]:
datos_hoy = datos_row.copy()

renombramos columnas, añadimos columna con `fecha_de_informe` y ordenamos

In [8]:
datos_hoy.columns=['ccaa','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [9]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de 'Fecha ultima vacuna' en %d/%m/%y

In [10]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [11]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

y asegurar que los tags de comunidades no tienes espacios:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

eliminamos la fila Totales

In [14]:
#datos_hoy = datos_hoy.drop(datos_hoy.index[-1])

In [15]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [16]:
datos_hoy

,date_pub,ccaa,Fecha de la ultima vacuna registrada,Dosis entregadas,Dosis administradas,% sobre entregadas,source_name,source
0,8/1/21,Andalucía,7/1/21,140295,69445,0.494993,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,8/1/21,Aragón,8/1/21,23715,9944,0.419313,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,8/1/21,Asturias,8/1/21,23720,17843,0.752234,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,8/1/21,Baleares,7/12/21,12840,4257,0.331542,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,8/1/21,Canarias,8/1/21,27660,9572,0.346059,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,8/1/21,Cantabria,8/1/21,11850,2334,0.196962,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,8/1/21,Castilla y Leon,7/1/21,51390,28325,0.551177,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,8/1/21,Castilla La Mancha,7/1/21,35550,12380,0.348242,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,8/1/21,Cataluña,7/1/21,120545,38826,0.322087,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,8/1/21,C. Valenciana,8/1/21,61225,15965,0.260759,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [17]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [ ]:
datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [18]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [19]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
5,4/1/21,Cantabria,11850,304,0.025654,30/12/20,30/12/20,Ministerio de Sanidad - Vacuna Covid 24,https://www.mscbs.gob.es/profesionales/saludPu...
6,4/1/21,Castilla y Leon,51390,10928,0.212648,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 25,https://www.mscbs.gob.es/profesionales/saludPu...
7,4/1/21,Castilla La Mancha,35550,1983,0.055781,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 26,https://www.mscbs.gob.es/profesionales/saludPu...
8,4/1/21,Cataluña,120545,8150,0.067610,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 27,https://www.mscbs.gob.es/profesionales/saludPu...
9,4/1/21,C. Valenciana,61225,3252,0.053116,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 28,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [21]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
75,8/1/21,Navarra,11850,3703,0.312489,NaN,7/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
76,8/1/21,País Vasco,31605,9811,0.310426,NaN,7/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
77,8/1/21,Ceuta,1005,637,0.633831,NaN,7/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
78,8/1/21,Melilla,1005,673,0.669652,NaN,8/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [23]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [24]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)